<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Evaluating a Multi-Object Tracking Model on MOT Challenge

This notebook provides a framework for evaluating [FairMOT](https://github.com/ifzhang/FairMOT) on the [MOT Challenge dataset](https://motchallenge.net/).

The MOT Challenge datasets are some of the most common benchmarking datasets for measuring multi-object tracking performance on pedestrian data. They provide distinct datasets every few years; their current offerings include MOT15, MOT16/17, and MOT19/20. These datasets contain various annotated video sequences, each with different tracking difficulties. Additionally, the MOT Challenge provides detections for tracking algorithms without detection components.

The goal of this notebook is to reproduce published results on the MOT Challenge using the state-of-the-art FairMOT approach.

## Initialization

In [1]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import os.path as osp
import sys
import time

from urllib.parse import urljoin
import torch
import torchvision

sys.path.append("../../")
from utils_cv.common.data import data_path, download, unzip_url
from utils_cv.common.gpu import which_processor, is_windows
from utils_cv.tracking.data import Urls
from utils_cv.tracking.dataset import TrackingDataset
from utils_cv.tracking.model import TrackingLearner

# Change matplotlib backend so that plots are shown for windows
if is_windows():
    plt.switch_backend("TkAgg")

print(f"TorchVision: {torchvision.__version__}")
which_processor()

TorchVision: 0.4.0a0
Torch is using GPU: Tesla K80


The above torchvision command displays your machine's GPUs (if it has any) and the compute that `torch/torchvision` is using.

Next, we will set some model runtime parameters. Here we specify the default FairMOT model that we will evaluate against the MOT17 dataset. The baseline FairMOT model is downloaded and saved to the `./models/baselines` directory as `all_dla34.pth` by running the below cell.

In [3]:
CONF_THRES = 0.4
TRACK_BUFFER = 30

# Downloaded MOT Challenge data path
MOT_ROOT_PATH = "../../data/"
RESULT_ROOT = "./results"
EXP_NAME = "MOT_val_all_dla34"

# Pretrained model location
MODEL_DIR = unzip_url(Urls.baseline_models_path, "./models", exist_ok=True)
BASELINE_MODEL = osp.join(MODEL_DIR, "all_dla34.pth")

# Train on the GPU or on the CPU, if a GPU is not available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using torch device: {device}")

Using torch device: cuda


## Model and Dataset Initialization

Now, we will download the [MOT17](https://motchallenge.net/data/MOT17.zip) dataset and save it to `MOT_SAVED_PATH`. 

Note: We extract a subset of MOT17 which is around 2 GB. It may take some time to download.

In [4]:
mot_path = unzip_url(Urls.mot_challenge_path, MOT_ROOT_PATH, exist_ok=True)
mot_train_path = osp.join(mot_path, "train")
mot_test_path = osp.join(mot_path, "test")
# seqs_str:  various video sequences subfolder names under MOT challenge data
train_seqs = [
    "MOT17-02-SDP",
    "MOT17-04-SDP",
    "MOT17-05-SDP",
    "MOT17-09-SDP",
    "MOT17-10-SDP",
    "MOT17-11-SDP",
    "MOT17-13-SDP",
]
test_seqs = [
    "MOT17-01-SDP",
    "MOT17-03-SDP",
    "MOT17-06-SDP",
    "MOT17-07-SDP",
    "MOT17-08-SDP",
    "MOT17-12-SDP",
    "MOT17-14-SDP",
]

print(f"Training data saved to {mot_train_path}")
print(f"Test data saved to {mot_test_path}")

Training data saved to /home/albuendi/computervision-recipes/data/MOT17/train
Test data saved to /home/albuendi/computervision-recipes/data/MOT17/test


The code below initializes and loads the model using the `TrackingLearner` class.

In [5]:
tracker = TrackingLearner(None, BASELINE_MODEL)

## Evaluate on Training Set

MOT17 provides ground truth annotations for only the training set, so we will be using the training set for evaluation.

To evaluate FairMOT on this dataset, we take advantage of the [py-motmetrics](https://github.com/cheind/py-motmetrics) repository. 

In [6]:
strsummary = tracker.eval_mot(
    conf_thres=CONF_THRES,
    track_buffer=TRACK_BUFFER,    
    data_root=mot_train_path,
    seqs=train_seqs,
    result_root=RESULT_ROOT,
    exp_name=EXP_NAME,
)
print(strsummary)

loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-02-SDP.txt
Evaluate seq: MOT17-02-SDP
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-04-SDP.txt
Evaluate seq: MOT17-04-SDP
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-05-SDP.txt
Evaluate seq: MOT17-05-SDP
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-09-SDP.txt
Evaluate seq: MOT17-09-SDP
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-10-SDP.txt
Evaluate seq

## Evaluate on Test Set

For evaluating a model on the testing dataset, the MOT Challenge provides the [MOT evaluation server](https://motchallenge.net/instructions/). Here, a user can upload and submit a `.txt` file of prediction results; the service will return metrics. After uploading our results to the [MOT17 evaluation server](https://motchallenge.net/results/MOT17/?det=Private), we can see a MOTA of 68.5 using the `all_dla34.pth` baseline model.

<img src="media/mot_results.PNG" style="width: 737.5px;height: 365px"/>

The reported evaluation results from the [FairMOT paper](https://arxiv.org/abs/2004.01888) with the test set are as follows:

| Dataset | MOTA   | IDF1 | IDS | MT | ML | FPS |
|------|------|------|------|------|------|------|
|   MOT16  | 68.7| 70.4| 953| 39.5%| 19.0%| 25.9|
|   MOT17  | 67.5| 69.8| 2868| 37.7%| 20.8%| 25.9|

In [7]:
tracker.eval_mot(
    conf_thres=CONF_THRES,
    track_buffer=TRACK_BUFFER,    
    data_root=mot_test_path,
    seqs=test_seqs,
    result_root=RESULT_ROOT,
    exp_name=EXP_NAME,
    run_eval=False,
)

loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-01-SDP.txt
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-03-SDP.txt
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-06-SDP.txt
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-07-SDP.txt
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-08-SDP.txt
loaded /home/albuendi/computervision-recipes/scenarios/tracking/models/baselines/all_dla34.pth, epoch 10
Saved tracking 